#### Model the impact of variations in pixel-to-pixel sensitivity on centroiding accuracy for a DeMi Shack Hartmann Sensor lenslet.

MT9P031 Testing for PRNU and radiation impact:

Becker, Heidi N., Michael D. Dolphin, Dennis O. Thorbourn, James W. Alexander, and Phil M. Salomon. Commercial Sensory Survey Radiation Testing Progress Report. Pasadena, CA: Jet Propulsion Laboratory, National Aeronautics and Space Administration, 2008.
    https://trs.jpl.nasa.gov/bitstream/handle/2014/40825/08-22.pdf?sequence=1
    
WFPC2 CCD:
Trauger, John T., Gilda E. Ballester, Christopher J. Burrows, Stefano Casertano, John T. Clarke, David Crisp, Robin W. Evans, et al. “The on-Orbit Performance of WFPC2.” The Astrophysical Journal Letters 435 (November 1, 1994): L3–6. doi:10.1086/187580.




In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import poppy
from DeMiOpticTools import resample_PSF #flux conserving PSF downsampling

import importlib
import DeMiOpticTools

import astropy.units as u
from poppy.sub_sampled_optics import subapertures
from matplotlib.colors import LogNorm
import scipy.ndimage.measurements as meas


import importlib
importlib.reload(poppy)

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

/Users/edouglas/anaconda/lib/python3.5/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/Users/edouglas/anaconda/lib/python3.5/site-packages/pysynphot/locations.py:329: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


ImportError: bad magic number in 'poppy.sub_sampled_optics': b'\x03\xf3\r\n'

setup case of multi DM using the SH design in A. Marinan's `AO_Spacecraft_Requirements_v10.xlsx` where 0.016 pix centroiding error.



In [ ]:
fl_primary=100*u.mm
fl_coll=15*u.mm
dm_size = 4.95*u.mm
wavel = 635*u.nm

m = fl_primary/fl_coll

pixel_size = 2.2*u.um/u.pix
lenslet_pitch = 150*u.um
r_lenslet = lenslet_pitch/2.
lenslet_focal_length = 3.7*u.mm #
plate_scale =1.0*u.rad/lenslet_focal_length*m
rad_pix = (plate_scale*pixel_size)
diff_lim = 1.22*wavel/(2*r_lenslet)*u.radian*m
#calculate plate scale:
print("calculated plate scale: "+str(rad_pix.to(u.arcsecond/u.pix)))
print("diff limit: "+str(diff_lim.to(u.arcsec)))
print(" pixels/PSF: "+str((diff_lim/rad_pix).to(u.pix)))

In [ ]:
(diff_lim*lenslet_focal_length /pixel_size).decompose()#to(u.m*u.rad)




In [ ]:
rad_pix.to(u.arcsecond/u.pixel)

In [ ]:
osys = poppy.OpticalSystem(oversample=10)
osys.add_pupil(poppy.CircularAperture(radius=lenslet_pitch.to(u.m).value/2.))
osys.add_detector(rad_pix.to(u.arcsec/u.pix), fov_pixels=16,oversample=8)
single_psf=osys.calc_psf()



plt.figure()
poppy.utils.display_psf(single_psf,interpolation='nearest')#,imagecrop=1000)

plt.figure()
poppy.display_PSF(resample_PSF(single_psf),interpolation='nearest')#,imagecrop=1000)
single_psf=resample_PSF(single_psf)

In [ ]:
#generate a realistitic QE map:
QE_map = np.random.normal(1, 0.02, single_psf[0].data.shape)
plt.imshow(QE_map,interpolation='nearest')
plt.colorbar()



In [ ]:
import astropy.io.fits as fits
single_psf[0].header["PIXSCALE"]=1
#noiseless_cen=np.array(poppy.utils.measure_centroid(single_psf))
noiseless_cen_cm=np.array(meas.center_of_mass(single_psf[0].data))
#print(noiseless_cen_cm-noiseless_cen)
def RSS(array):
    return np.sqrt(np.sum(array**2))
sigma_QE_range=np.arange(0.005,0.08,0.01)
ax=plt.subplot(111)
for run in range(10):
    centroid_err=np.zeros(sigma_QE_range.size)
    centroid_err_cm=np.zeros(sigma_QE_range.size)


    for i,sigma_QE in enumerate(sigma_QE_range):
        QE_map = np.random.normal(1, sigma_QE, single_psf[0].data.shape)
        laser_electrons=2489210820.49 #electrons/second? CHECK
        PSF_electrons = single_psf[0].data*laser_electrons/single_psf[0].data.sum()
        psf_QE = fits.HDUList([fits.PrimaryHDU(QE_map*PSF_electrons)])
        psf_QE[0].header["PIXSCALE"]=1
        
        #centroid_err[i]=RSS(noiseless_cen-np.array(poppy.utils.measure_centroid(psf_QE)))
    
        centroid_err_cm[i] = RSS(noiseless_cen_cm-np.array(meas.center_of_mass(psf_QE[0].data)))

    #plt.plot(sigma_QE_range,centroid_err)
    plt.plot(sigma_QE_range,centroid_err_cm,'--')
plt.plot([0.01,0.01],[0,2],label="WFPC2 QE stability",linewidth=2)

plt.plot([0.01,0.015],[0,2],
         label="MT9P031 PRNU",linewidth=2)
plt.plot([0.01,0.027],[0,2],
         label="MT9P031 PRNU -1 krad",linewidth=3)
plt.plot([0.01,0.045],[0,2],
         label="MT9P031 PRNU -5 krad",linewidth=4)

plt.grid(True)



plt.ylim([1e-3,1])
plt.xlabel("$\sigma_{QE}$")
plt.yscale("log")

plt.ylabel("centroid error [pix]")
plt.legend(loc="upper left")

In [ ]:
import astropy.io.fits as fits
plt.figure(figsize=[5,4])
single_psf[0].header["PIXSCALE"]=1
#noiseless_cen=np.array(poppy.utils.measure_centroid(single_psf))
noiseless_cen_cm=np.array(meas.center_of_mass(single_psf[0].data))
#print(noiseless_cen_cm-noiseless_cen)
def RSS(array):
    return np.sqrt(np.sum(array**2))
sigma_QE_range=np.arange(0.001,.1,0.005)
ax=plt.subplot(111)
for run in range(100):
    centroid_err=np.zeros(sigma_QE_range.size)
    centroid_err_cm=np.zeros(sigma_QE_range.size)


    for i,sigma_QE in enumerate(sigma_QE_range):
        QE_map = np.random.normal(1, sigma_QE, single_psf[0].data.shape)
        laser_electrons=2489210820.49 #electrons/second? CHECK
        PSF_electrons = single_psf[0].data*laser_electrons/single_psf[0].data.sum()
        psf_QE = fits.HDUList([fits.PrimaryHDU(QE_map*PSF_electrons)])
        psf_QE[0].header["PIXSCALE"]=1
        
        #centroid_err[i]=RSS(noiseless_cen-np.array(poppy.utils.measure_centroid(psf_QE)))
    
        centroid_err_cm[i] = RSS(noiseless_cen_cm-np.array(meas.center_of_mass(psf_QE[0].data)))

    #plt.plot(sigma_QE_range,centroid_err)
    plt.plot(sigma_QE_range,centroid_err_cm,alpha=.25)
#plt.plot([0.01,0.01],[0,2],label="WFPC2 QE stability",linewidth=2)

plt.plot([0.01,0.0151],[0,2],'--',
         label="MT9P031 PRNU",linewidth=1)
plt.plot([0.01,0.021],[0,2],'-.',
         label="MT9P031 PRNU 0.5 krad(Si)",linewidth=2)
plt.plot([0.01,0.027],[0,2],'--',
         label="MT9P031 PRNU 1 krad(Si)",linewidth=3)
plt.plot([0.01,0.045],[0,2],'--',
         label="MT9P031 PRNU 5 krad(Si)",linewidth=4)
requirement=(10*u.nm*lenslet_focal_length/lenslet_pitch/pixel_size).decompose()

plt.grid(True)

def tick_function(Y):
    #https://stackoverflow.com/questions/10514315/how-to-add-a-second-x-axis-in-matplotlib
    V =(Y*u.pix/(lenslet_focal_length/lenslet_pitch/pixel_size)).to(u.nm).value
    return ["%.e" % z for z in V]


plt.ylim([1e-4,1])
plt.xlabel("$\sigma_{QE}$")
plt.yscale("log")

plt.ylabel("centroid error [pix]")
plt.legend(loc="upper right")
ax2=ax.twinx()
ax2.set_yticklabels(tick_function(ax.get_yticks()))
#https://stackoverflow.com/questions/44080894/matplotlib-tick-get-loc-always-returns-zero#44084174
ax2.set_ylabel("Wavefront Tilt [nm/lenslet]")

plt.savefig("PRNU_centroid_error.pdf",bbox_inches="tight")

In [ ]:
(10*u.nm*lenslet_focal_length/lenslet_pitch/pixel_size).decompose()#.to(u.m)


Annie's spreadsheet calls for 0.016-0.036 pixel centroiding, the former is challenging and the latter is doable if flat field error is the only source of centroid error.

Conclusions:
* the impact of interpixel QE variation above a percent is non-neglible at the required wavefront centroiding precisions
* radiation shielding matters 
* detector flat fielding in flight would be useful 
* the result doesnt appear very sensitive to PSF size


## Add detector noise and calculate for star and laser

In [ ]:
DarkCurrent_aptina=25#*u.electron/u.second #http://www.mouser.com/ds/2/308/LimitedDataSheet_MT9P031_5100_PB.book-553282.pdf
RN_aptina = 2.5#*u.electron #http://digitalcommons.usu.edu/cgi/viewcontent.cgi?article=1162&context=smallsat
t_exp=1/12.#*u.second
second_mag_star=70/t_exp#.1876429374  #electrons/sec
plt.figure(figsize=[7,3])
plt.subplot(121)
importlib.reload(DeMiOpticTools)
PSF_electrons = single_psf[0].data*second_mag_star/single_psf[0].data.sum()
PSF_electrons_HDU= fits.HDUList([fits.PrimaryHDU(PSF_electrons,
                                                 header=single_psf[0].header)])
#electrons/s from 2 mag star

noisyPSF=[DeMiOpticTools.simulate_noise(PSF_electrons_HDU,
                                       n_exp=1,
                                       dark_noise_rate=DarkCurrent_aptina,
                                       read_noise=RN_aptina,
                                       t_exp=one*t_exp) for one in np.ones(1000)] 
plt.imshow(noisyPSF[0][0].data,interpolation='none',cmap='plasma')
plt.colorbar()

star_deltaCM= [RSS(noiseless_cen_cm-np.array(meas.center_of_mass(psf[0].data))) for psf in noisyPSF]
star_delta_nm=(np.sin((star_deltaCM*u.pix*pixel_size*plate_scale/m))*lenslet_pitch).to(u.nm)

plt.subplot(122)
plt.hist(star_delta_nm,bins='auto')
plt.xscale("log")
plt.title("median\n"+str(np.round(np.median(star_delta_nm))))#,star_delta_nm.std()

In [ ]:
plate_scale/m

In [ ]:
948*10**(+.02/2.5)

In [ ]:
laser_electrons=2489210820.49 #electrons/second? CHECK
importlib.reload(DeMiOpticTools)
PSF_electrons = single_psf[0].data*laser_electrons/single_psf[0].data.sum()
PSF_electrons_HDU= fits.HDUList([fits.PrimaryHDU(PSF_electrons,header=single_psf[0].header)])
#electrons/s from 2 mag star

noisyPSF=DeMiOpticTools.simulate_noise(PSF_electrons_HDU,
                                       n_exp=1,
                                       dark_noise_rate=25,
                                       read_noise=2.5,
                                       t_exp=1.)
plt.imshow(noisyPSF[0].data)
plt.colorbar()


laser_deltaCM= RSS(noiseless_cen_cm-np.array(meas.center_of_mass(noisyPSF[0].data)))

star_deltaCM= RSS(noiseless_cen_cm-np.array(meas.center_of_mass(noisyPSF[0].data)))
(np.sin((star_deltaCM*u.pix*pixel_size*plate_scale))*lenslet_pitch).to(u.nm)


In [ ]:
print(len(sigma_QE_range))